# Tabular Playground Series - Feb 2021

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Read datasets

In [2]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv', index_col='id')
display(train.sample(2))

FileNotFoundError: [Errno 2] No such file or directory: '../input/tabular-playground-series-feb-2021/train.csv'

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv', index_col='id')
display(test.sample(2))

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv', index_col='id')
display(submission.sample(2))

In [ ]:
X = train.drop(['target'], axis=1)
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state = 42)

## EDA

## LazyPredict

In [ ]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -q lazypredict
from lazypredict.Supervised import LazyRegressor

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models,predictions = reg.fit(X_train, X_test, y_train, y_test)

In [ ]:
print(models)

## AutoGluon

In [ ]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -U -q "mxnet<2.0.0"
!pip install -q autogluon
from autogluon.tabular import TabularPredictor

In [ ]:
predictor = TabularPredictor(label='target', 
                             problem_type='regression', 
                             eval_metric='root_mean_squared_error'
                             ).fit(train, 
                                   #time_limit=840, 
                                   presets='high_quality_fast_inference_only_refit'
                                   )

In [ ]:
predictor.leaderboard()

In [ ]:
submission['target'] = predictor.predict(test)
submission.to_csv('../output/submission_autogluon.csv')

## EvalML

In [ ]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -q evalml
import evalml
from evalml.automl import AutoMLSearch

In [ ]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='regression')

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='regression')
automl.search()

In [ ]:
automl.rankings

In [ ]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

In [ ]:
best_pipeline.score(X_test, y_test)

## PyCaret